# Крыжановский Максим, 317

**Алгоритмы из курса методы оптимизации**

In [2]:
import numpy as np
from scipy.optimize import LinearConstraint as LC
from scipy.optimize import NonlinearConstraint as NC
from scipy.optimize import minimize
from scipy.optimize import linprog

# Метод проекции градиента

Рассмотрим задачу минимизации

$$J(u) = \|u\|^2_{\mathbb{H}} + \left<u,a\right>^2_{\mathbb{H}} + \left<u,b\right>_{\mathbb{H}} \rightarrow \inf_{u \in U},  \|a\|_{\mathbb{H}} = \|b\|_{\mathbb{H}} = 1, \langle a, b\rangle_{\mathbb{H}} = 0$$

$$ U = \{u \in \mathbb{H} \ | \ \|u - u_0\|_{\mathbb{H}} \leq R \}$$

In [3]:
class FunctionOracle:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, u):
        return np.linalg.norm(u) ** 2 + np.dot(u,self.a) ** 2 + np.dot(u,self.b)
    
    def grad(self, u):
        return 2 * u + 2 * np.dot(u, self.a) * self.a + self.b

In [8]:
class Solver:
    def __init__(self, oracle, u0, R, learning_rate=0.01, tolerance=1e-6, max_iterations=1000):
        self.oracle = oracle
        self.u0 = u0
        self.R = R
        self.learning_rate = learning_rate
        self.tolerance = tolerance
        self.max_iterations = max_iterations
        
    def project(self, u):
        norm_diff = np.linalg.norm(u - self.u0)
        if norm_diff > self.R:
            u = self.u0 + self.R * (u - self.u0) / norm_diff
        return u
    
    def solve(self, eps=1e-8):
        u_k = self.u0
        iterations = 0
        u_list = []

        while True:
            projection = self.project(u_k - self.learning_rate * self.oracle.grad(u_k))

            u_list.append(u_k)
            u_prev, u_k = u_k, projection

            iterations += 1
            if np.linalg.norm(u_k - u_prev) < eps:
                break

        return u_k, iterations, u_list

**Пример импользования кода**

В примере использования, сравним результаты, полученные нашим алгоритмом, с результатами библиотеки scipy.

In [11]:
a = np.array([1, 0])
b = np.array([0, 1])
u0 = np.array([0.5, 0.5])
R = 1.0

oracle = FunctionOracle(a, b)

solver = Solver(oracle, u0, R)
res = solver.solve()
print("Custom Solver Solution:", res[0])
print("J value:", oracle(res[0]))

def objective(u):
    return oracle(u)

def gradient(u):
    return oracle.grad(u)
t
constraints = {'type': 'ineq', 'fun': lambda u: R - np.linalg.norm(u - u0)}

result = minimize(objective, u0, method='SLSQP', jac=gradient, constraints=[constraints])
solution_scipy = result.x
print("Scipy Minimize Solution:", solution_scipy)
print("J value:", oracle(solution_scipy))

Custom Solver Solution: [ 0.03100984 -0.38320339]
J value: -0.23443533176574022
Scipy Minimize Solution: [ 0.03101152 -0.38320473]
J value: -0.23443543646034093


# Метод Ньютона

В этом разделе реализуем метод Ньютона для следующей задачи

$$J(u) = \frac{1}{2} \|u-a\|^4_{\mathbb{H}} + \|u-b\|^2_{\mathbb{H}} \rightarrow \inf_{u \in U},  U=\{u \in \mathbb{H} | \left< a+b, u\right>_{\mathbb{H}} \geq 2\}, \|a\|_{\mathbb{H}} = \|b\|_{\mathbb{H}} = 1, \left< a, b\right>_{\mathbb{H}} = 0$$

In [12]:
class FunctionOracle():
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, u):
        return 1/2 * (np.linalg.norm(u - self.a) ** 4) + np.linalg.norm(u - self.b) ** 2
    
    def grad(self, u):
        return 2 * (np.linalg.norm(u - self.a) ** 2) * (u - self.a) + 2 * (u - self.b)
    
    def hess(self, u, h):
        return 4 * np.dot(u - self.a, h) * (u - self.a) + 2 * ((np.linalg.norm(u-self.a) ** 2) + 1) * h

In [15]:
class SolverNewton:
    def __init__(self, oracle, u0, tolerance=1e-6, max_iterations=100):
        self.oracle = oracle
        self.u0 = u0
        self.tolerance = tolerance
        self.max_iterations = max_iterations
        
    def solve(self):
        u_list = []
        u_k = self.u0
        iterations = 0

        while True:
            u_list.append(u_k)
            u_prev, u_k = u_k, self.gradient_projection(self.grad_G(u_k), self.projector, self.u0, 1/4, eps=self.tolerance)[0]

            iterations += 1
            if np.linalg.norm(u_k - u_prev) < self.tolerance:
                break
        return u_k, iterations, u_list
    
    def gradient_projection(self, grad_J, projector, u_0, alpha, eps=1e-8):
        u_k = u_0
        iterations = 0
        u_list = []

        while True:
            projection = projector(u_k - alpha * grad_J(u_k))

            u_list.append(u_k)
            u_prev, u_k = u_k, projection

            iterations += 1
            if np.linalg.norm(u_k - u_prev) < eps:
                break

        return u_k, iterations, u_list
    
    def grad_G(self, u_k):
        def _inner(u):
            return self.oracle.grad(u_k) + 4 * (u_k - self.oracle.a) * np.dot(u_k - self.oracle.a, u - u_k) + 2 * (np.linalg.norm(u_k - self.oracle.a) ** 2 + 1) * (u - u_k)
        return _inner
    
    def projector(self, u):
        if np.dot(self.oracle.a + self.oracle.b, u) >= 2:
            return u
        return u - (np.dot(self.oracle.a + self.oracle.b, u) - 2) * (self.oracle.a + self.oracle.b) / (np.linalg.norm(self.oracle.a + self.oracle.b) ** 2)

In [17]:
a = np.array([1.0, 0.0])
b = np.array([0.0, 1.0])
u0 = np.array([0.5, 0.5])

oracle = FunctionOracle(a, b)
solver_newton = SolverNewton(oracle, u0)
solution_newton = solver_newton.solve()
print("Newton Solver Solution:", solution_newton[0])
print("J value:", oracle(solution_newton[0]))

# Define the objective function and gradient for scipy.optimize.minimize
def objective(u):
    return oracle(u)

def gradient(u):
    return oracle.grad(u)

# Use scipy.optimize.minimize with 'Newton-CG' method
result = minimize(objective, u0, method='SLSQP', constraints=LC(a + b, 2, np.inf))
solution_scipy = result.x
print("Scipy Minimize Solution:", solution_scipy)
print("J value:", oracle(solution_scipy))

Newton Solver Solution: [1.00000012 0.99999988]
Scipy Minimize Solution: [0.99998258 1.00001742]
